# 📜 Tratamento dos dados entre videos do Youtube e os stocks de grandes empresas 📜 

## Importar Bibliotecas

In [24]:
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark import SparkContext
from datetime import datetime
import re
from functools import reduce
from matplotlib import cbook, cm
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt

## Criação do Spark para ser utilizado em PySpark

In [25]:
spark_ = SparkSession.builder \
    .appName("company_database") \
    .config("spark.mongodb.input.uri", "mongodb://mongodb:27017/Company_Database") \
    .config("spark.mongodb.output.uri", "mongodb://mongodb:27017/Company_Database") \
    .config("spark.mongodb.input.uri", "mongodb://mongodb:27017/Youtube_Database") \
    .config("spark.mongodb.output.uri", "mongodb://mongodb:27017/Youtube_Database") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.driver.memory", "7g") \
    .config("spark.executor.memory", "7g") \
    .getOrCreate()

### Datasets
Para a realização deste trabalho, foram utilizados 14 datasets que se dividem em dois grupos principais: os datasets de vídeos do YouTube e datasets de stocks de empresas.

## Importanção dos dados de cada pais
Datasets de vídeos do youtube <br>
Os dados destes datasets são provenientes de oito diferentes países, dos quais: Brasil, Estados Unidos (US), Canadá, França, Inglaterra, Índia, Coreia do Sul e México.

In [26]:
df_us = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_us").load()
df_br = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_br").load()
df_ca = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_ca").load()
df_fr = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_fr").load()
df_gb = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_gb").load()
df_in = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_in").load()
df_kr = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_kr").load()
df_mx = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_mx").load()

## Importanção dos dados de empresa
Datasets de stocks de empresas <br>
Os datasets sobre os stocks de empresas fornecem dados de seis organizações distintas: Sony, Nvidia, Microsoft, Dell, IBM, Intel

In [27]:
df_dell = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Company_Database.dell_data").load()
df_ibm = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Company_Database.ibm_data").load()
df_intel = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Company_Database.intel_data").load()
df_microsoft = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Company_Database.microsoft_data").load()
df_nvidia = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Company_Database.nvidia_data").load()
df_sony = spark_.read.format("mongo").option("uri", "mongodb://mongodb:27017/Company_Database.sony_data").load()

## Dicionario das empresas e seus nomes para posteriormente fazer a união dos dfs com a coluna company_name
A ideia é juntar todos os dataframes num só. Para isso, necessitamos de criar uma nova coluna que identifique a que empresa a linha do stock está ligada

In [28]:
dict_companies = {'companies': [
                 {'dataframe':df_dell, 'name': 'dell'},
                 {'dataframe':df_ibm, 'name': 'ibm'},
                 {'dataframe':df_intel, 'name': 'intel'},
                 {'dataframe':df_microsoft, 'name': 'microsoft'},
                 {'dataframe':df_nvidia, 'name': 'nvidia'},
                 {'dataframe':df_sony, 'name': 'sony'},  
                ]
            }

In [29]:
for company in dict_companies['companies']:
    company["dataframe"] = company["dataframe"].withColumn("company_name", lit(company["name"]))

In [30]:
df_companies = dict_companies['companies'][0]['dataframe'].union(dict_companies['companies'][1]['dataframe']).union(dict_companies['companies'][2]['dataframe']).union(dict_companies['companies'][3]['dataframe']).union(dict_companies['companies'][4]['dataframe']).union(dict_companies['companies'][5]['dataframe'])

## Filtro dos dados das varias empresas

Filtragem de Dados: os dados foram filtrados com base no título do vídeo e no nome do canal em cada região, mantendo apenas aqueles que mencionam diretamente as empresas alvo. Esta etapa foi essencial para garantir que os dados refletiam a relação com as empresas analisadas.

O filtro foi realizado nas colunas title e channelTitle, enquanto as demais colunas foram desconsideradas devido ao grande volume de ruído. Um exemplo claro disso são as colunas de tags e comentários. O ruído ocorre porque vídeos postados por pessoas que buscam alcançar um público mais amplo frequentemente incluem tags e informações com palavras-chave relacionadas a empresas ou áreas com grande número de interessados. Isso faz com que o algoritmo do YouTube mostre esses vídeos com mais frequência para utilizadores que demonstram interesse nessas palavras-chave.

In [31]:
companies = ['nvidia', 'dell', 'ibm', 'intel', 'microsoft', 'sony']
countries_df = [df_us, df_br, df_ca, df_fr, df_gb, df_in, df_kr, df_mx]
countries = ['us', 'br', 'ca', 'fr', 'gb', 'in', 'kr', 'mx']

yt = {company: {} for company in companies}

def filter_df(df, company):
    filtered_df = df.filter(
        (col('title').contains(company.capitalize())) |
        (col('title').contains(company.upper())) |
        (col('channelTitle').contains(company.capitalize())) |
        (col('channelTitle').contains(company.upper()))
    )
    return filtered_df.withColumn("company", lit(company))

for company in companies:
    for country, country_df in zip(countries, countries_df):
        filtered_data = filter_df(country_df, company)
        yt[company].setdefault(country, []).append(filtered_data)


## União dos df de cada pais num só
Para facilmente ser analisado os dados de cada empresa, juntou se todos os dados num so dataframe


In [32]:
all_dfs = [
    df 
    for company, country_data in yt.items() 
    for country, dfs in country_data.items() 
    for df in dfs
]

if all_dfs:
    df_yt = reduce(lambda df1, df2: df1.union(df2), all_dfs)
else:
    df_yt = None  


## Colunas existentes

In [33]:
#df_yt.printSchema()

In [34]:
#df_companies.count()

In [35]:
#df_yt.count()

In [36]:
#df_companies.count()

## União dos videos de cada empresa
df_yt_companies - todos os videos das empresas a serem estudadas<br> 
df_all_companies - dados de todas as empreas a serem estudadas

## Estudo dos nulos e duplicados

A existencia de nulos em analises de dados é um problema que tem de ser sempre tratado. O mesmo em relação aos duplicados (um problema que tinhamos em grandes quantidades)

In [37]:
if df_yt is not None:
    df_yt.select([count(when(col(c).isNull(), c)).alias(c) for c in df_yt.columns]).show()
else:
    print("df_yt is None")

+---+----------+---------+------------+-------------+-----------------+-----------+--------+-----+-----------+----------------+----+--------------+-----+-------------+--------+----------+-------+
|_id|categoryId|channelId|channelTitle|comment_count|comments_disabled|description|dislikes|likes|publishedAt|ratings_disabled|tags|thumbnail_link|title|trending_date|video_id|view_count|company|
+---+----------+---------+------------+-------------+-----------------+-----------+--------+-----+-----------+----------------+----+--------------+-----+-------------+--------+----------+-------+
|  0|         0|        0|           0|            0|                0|          0|       0|    0|          0|               0|   0|             0|    0|            0|       0|         0|      0|
+---+----------+---------+------------+-------------+-----------------+-----------+--------+-----+-----------+----------------+----+--------------+-----+-------------+--------+----------+-------+



In [38]:
duplicates = df_yt.groupBy("title").count().filter("count > 1")
duplicates.show()

+--------------------+-----+
|               title|count|
+--------------------+-----+
|EVGA Terminates N...|   18|
|I've never held s...|    9|
|BREAKING NEWS! - ...|   15|
|NVIDIA... You've ...|   18|
|NVIDIA GeForce RT...|   15|
|Nvidia was clearl...|   19|
|ITS HERE - NVIDIA...|   16|
|NVIDIA finally an...|    5|
|NVIDIA REFUSED To...|   14|
|NVIDIA GeForce RT...|   13|
|NVIDIA just made ...|   22|
|Nvidia tried to b...|    8|
|NVIDIA 40 Series ...|    9|
|Nvidia, you PROMI...|   14|
|NVIDIA Never saw ...|   15|
|NVIDIA expects to...|    4|
|NVIDIA RTX 3090, ...|   13|
|NVIDIA... why do ...|   18|
|NVIDIA's RTX 4080...|   28|
|Black Myth: Wukon...|   14|
+--------------------+-----+
only showing top 20 rows



In [39]:
df_yt = df_yt.dropDuplicates(["title"])

In [40]:
df_companies.select([count(when(col(c).isNull(), c)).alias(c) for c in df_companies.columns]).show()

+---------+-----+----+----+---+----+------+---+------------+
|Adj Close|Close|Date|High|Low|Open|Volume|_id|company_name|
+---------+-----+----+----+---+----+------+---+------------+
|        0|    0|   0|   0|  0|   0|     0|  0|           0|
+---------+-----+----+----+---+----+------+---+------------+



In [41]:
duplicates = df_companies.groupBy("Date").count().filter("count > 1")
duplicates.show()

df_cleaned = df_companies.dropDuplicates(["Date"])

+----------+-----+
|      Date|count|
+----------+-----+
|2016-08-17|    6|
|2017-12-05|    6|
|2019-08-08|    6|
|2019-08-22|    6|
|2019-08-23|    6|
|2020-02-26|    6|
|2020-04-13|    6|
|2021-11-03|    6|
|2022-10-05|    6|
|2023-05-01|    6|
|2023-05-18|    6|
|2024-01-19|    6|
|2024-08-20|    5|
|2024-10-24|    5|
|2017-02-24|    6|
|2017-05-11|    6|
|2017-10-20|    6|
|2017-12-22|    6|
|2018-12-31|    6|
|2019-04-25|    6|
+----------+-----+
only showing top 20 rows



## Passar publishedAt e Date de string para data

De forma a posteriormente ser utilizado nas visualizações

In [42]:
df_yt = df_yt.withColumn(
    "publishedAt",
    to_date(df_yt.publishedAt, "yyyy-MM-dd'T'HH:mm:ss'Z'")
)


In [43]:
df_companies = df_companies.withColumn(
    "Date",
    to_date(df_companies.Date, "yyyy-MM-dd")
)


## Tags String to Array sem nulos

para mais facilmente ser utilizado em termos de visualizações, transformamos a coluna das tags em um array de string. Posteriormente, se o array for nulo, a tag passa a ser o nome da companhia

In [44]:
df_transformed = df_yt.withColumn("tags_arr", split(col("tags"), "\|")).drop("tags")

Como o código estava antes, queríamos que funcionasse, mas o PySpark entrou em breakdwn e arrebentou com o kernel. Isso ocorreu porque não havia RAM suficiente (a RAM do Docker não permite mais do que 7 GB alocados no nosso contêiner).

Posteriormente, por não conseguirmos encontrar soluções na internet, decidimos deixar uma versão mais simplificada.

No código de filtro dos dados, o mesmo aconteceu quando tentamos usar regex para retirar mais facilmnte o 

```python
# no caso das tags
df_transformed = df_yt.withColumn(
    "tags_arr",
    when(col("tags").isNull() | (col("tags") == ""), col("company_name")) 
    .otherwise(split(col("tags"), "\|"))  
).drop("tags")) 

#no caso do regex

pattern = r'\b' + re.escape(company) + r'\b`

In [45]:
df_transformed.select("tags_arr").show(5)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o1987.showString

## Meter os dados filtrados na Database final do mongo

In [ ]:
df_yt.write.format("mongo") \
    .option("uri", "mongodb://mongodb:27017/Final_Database.youtube") \
    .mode("append") \
    .save()

df_companies.write.format("mongo") \
    .option("uri", "mongodb://mongodb:27017/Final_Database.company") \
    .mode("append") \
    .save()

print("Dados processados e salvos com sucesso.")
